<a href="https://colab.research.google.com/github/hail-members/llm-based-services/blob/main/Chapter_5_%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Train 데이터 다운로드
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json

# Dev 데이터 다운로드
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

import json

# Train 데이터 로드
with open("KorQuAD_v1.0_train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

# Dev 데이터 로드
with open("KorQuAD_v1.0_dev.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

# 데이터 구조 확인
print("Train Data Keys:", train_data.keys())
print("Example Data:", train_data["data"][0])  # 첫 번째 문단 출력

--2025-04-01 17:31:49--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   112MB/s    in 0.3s    

2025-04-01 17:31:53 (112 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2025-04-01 17:31:53--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [10]:
# 라이브러리 임포트
import torch
from transformers import GPT2LMHeadModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader


# KoGPT2 모델과 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [11]:
# 데이터셋 정의 (질문과 답변을 GPT2 입력 형식으로 변환)
class KorQuADDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data["data"]
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.total_qas = []  # 모든 질문-답변 쌍을 저장

        # 전체 질문-답변 쌍을 리스트로 저장
        for article in self.data:
            for paragraph in article["paragraphs"]:
                for qa in paragraph["qas"]:
                    self.total_qas.append((paragraph["context"], qa))

    def __len__(self):
        return len(self.total_qas)

    def __getitem__(self, idx):
        # 질문과 답변 추출
        context, qa = self.total_qas[idx]
        question = qa["question"]
        answer_text = qa["answers"][0]["text"]

        # GPT2 입력 텍스트 생성 (질문 + 컨텍스트 + 답변)
        input_text = f"<usr> {question} <sys> {context} </s>"
        target_text = f"<sys> {answer_text} </s>"

        # 토큰화 및 인코딩
        input_ids = self.tokenizer.encode(input_text, max_length=self.max_length, truncation=True, padding="max_length")
        target_ids = self.tokenizer.encode(target_text, max_length=self.max_length, truncation=True, padding="max_length")

        return {
            "input_ids": torch.tensor(input_ids),
            "labels": torch.tensor(target_ids)
        }

train_dataset = KorQuADDataset(train_data, tokenizer)
dev_dataset = KorQuADDataset(dev_data, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=4)


In [12]:
# 저장된 데이터 보기
for batch in train_dataloader:
    print(batch)
    break

{'input_ids': tensor([[    2,  9454, 25681,  ...,     3,     3,     3],
        [    2,  9034,  8529,  ...,     3,     3,     3],
        [    2,  9193,  8367,  ...,     3,     3,     3],
        [    2, 12375,  9656,  ...,     3,     3,     3]]), 'labels': tensor([[    4,  9752, 11996,  ...,     3,     3,     3],
        [    4, 39546, 10895,  ...,     3,     3,     3],
        [    4,  9040,  7198,  ...,     3,     3,     3],
        [    4,  9036,  8095,  ...,     3,     3,     3]])}


In [13]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 문장 생성 함수 정의
def generate_sentence(model, seed_text, max_length=50):
    input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(device)
    gen_ids = model.generate(
        input_ids,
        max_length=max_length,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        use_cache=True,
    )
    generated_text = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
    return generated_text

question = "인공지능이란?"
context = ""

# 입력 텍스트 생성
input_text = f"<usr> {question} <sys> {context} </s>"
input_ids = tokenizer.encode(
    input_text,
    max_length=100,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
).to(model.device)

# KoGPT2로 문장 생성
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=150,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print(f"질문: {question}")
print(f"컨텍스트: {context}")
print(f"생성된 답변: {generated_text}")


질문: 인공지능이란?
컨텍스트: 
생성된 답변: 인공지능이란?   
, , . (중략) 이번에 출시된 신제품은 '스마트폰용 스마트패드'다.
이 제품은 스마트폰을 통해 다양한 콘텐츠를 즐길 수 있는 것이 특징이다.
특히 기존 제품보다 최대 2배 이상 빠른 속도로 데이터를 전송


In [14]:

from torch.optim import AdamW
from tqdm import tqdm

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-3)

# 학습 루프 정의 (tqdm으로 Progress Bar 추가)
epochs = 1
model.train()

# 조기 종료 조건 설정
max_batches_per_epoch = 1000  # 한 에포크에서 최대 실행할 배치 수

for epoch in range(epochs):
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}", total=len(train_dataloader))

    for batch_idx, batch in progress_bar:
        if batch_idx >= max_batches_per_epoch:  # 조기 종료 조건
            print(f"Stopping early at batch {batch_idx} in epoch {epoch + 1}")
            break

        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix({"Batch Loss": loss.item()})

    print(f"Epoch {epoch + 1} completed. Average Loss: {epoch_loss / (batch_idx + 1)}")

# 모델 저장
model.save_pretrained("./kogpt2-korquad-finetuned")
tokenizer.save_pretrained("./kogpt2-korquad-finetuned")

Epoch 1:   7%|▋         | 1000/15102 [09:08<2:08:51,  1.82it/s, Batch Loss=0.0711]


Stopping early at batch 1000 in epoch 1
Epoch 1 completed. Average Loss: 0.2744662070332409


('./kogpt2-korquad-finetuned/tokenizer_config.json',
 './kogpt2-korquad-finetuned/special_tokens_map.json',
 './kogpt2-korquad-finetuned/vocab.json',
 './kogpt2-korquad-finetuned/merges.txt',
 './kogpt2-korquad-finetuned/added_tokens.json',
 './kogpt2-korquad-finetuned/tokenizer.json')

In [8]:

question = "인공지능이란?"
context = ""

# 입력 텍스트 생성
input_text = f"<usr> {question} <sys> {context} </s>"
input_ids = tokenizer.encode(
    input_text,
    max_length=100,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
).to(model.device)

# KoGPT2로 문장 생성
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=150,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print(f"질문: {question}")
print(f"컨텍스트: {context}")
print(f"생성된 답변: {generated_text}")

질문: 인공지능이란?
컨텍스트: 
생성된 답변: 인공지능이란?   


In [19]:

question = "인공지능이란?"
context = ""

# 입력 텍스트 생성
input_text = f"<usr> {question} <sys> {context} </s>"
input_ids = tokenizer.encode(
    input_text,
    max_length=5,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
).to(model.device)

# KoGPT2로 문장 생성
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=6,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print(f"질문: {question}")
print(f"컨텍스트: {context}")
print(f"생성된 답변: {generated_text}")

질문: 인공지능이란?
컨텍스트: 
생성된 답변: 인공지능이란
